In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.xception import Xception
import cv2

2023-03-07 14:53:23.054154: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
model = Xception(include_top=False)
for layer in model.layers:
    layer.trainable = False
model.summary()

83683744/83683744 [==============================] - 6s 0us/step
Model: "xception"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 block1_conv1 (Conv2D)          (None, None, None,   864         ['input_2[0][0]']                
                                32)                                                               
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, None, None,   128        ['block1_conv1[0][0]']           
 ation)                   

In [4]:
df = pd.read_parquet('data/1000035562.parquet')
y = pd.read_csv('data/train.csv')
df

,frame,row_id,type,landmark_index,x,y,z
0,20,20-face-0,face,0,0.494400,0.380470,-0.030626
1,20,20-face-1,face,1,0.496017,0.350735,-0.057565
2,20,20-face-2,face,2,0.500818,0.359343,-0.030283
3,20,20-face-3,face,3,0.489788,0.321780,-0.040622
4,20,20-face-4,face,4,0.495304,0.341821,-0.061152
...,...,...,...,...,...,...,...
12484,42,42-right_hand-16,right_hand,16,0.001660,0.549574,-0.145409
12485,42,42-right_hand-17,right_hand,17,0.042694,0.693116,-0.085307
12486,42,42-right_hand-18,right_hand,18,0.006723,0.665044,-0.114017
12487,42,42-right_hand-19,right_hand,19,-0.014755,0.643799,-0.123488


In [15]:
df.type.value_counts()

face          10764
pose            759
left_hand       483
right_hand      483
Name: type, dtype: int64

In [63]:
last = df.loc[(df.landmark_index == 0) & (df.frame == df.frame.iloc[-1])][['type','x','y']]
first = df.loc[(df.landmark_index == 0) & (df.frame == df.frame.iloc[0])][['type','x','y']]
display(last)
display(first)
last.x = last.x - first.x
display(last)

,type,x,y
11946,face,0.536416,0.375198
12414,left_hand,NaN,NaN
12435,pose,0.546659,0.343223
12468,right_hand,0.148969,0.757050


,type,x,y
0,face,0.494400,0.380470
468,left_hand,NaN,NaN
489,pose,0.524597,0.329134
522,right_hand,0.317082,0.576334


,type,x,y
11946,face,NaN,0.375198
12414,left_hand,NaN,NaN
12435,pose,NaN,0.343223
12468,right_hand,NaN,0.757050


In [66]:
df.loc[df.x.isna()].type.value_counts()

left_hand     483
right_hand    252
Name: type, dtype: int64

In [71]:
df.loc[df.type == 'right_hand'].loc[df.x.isna()]

,frame,row_id,type,landmark_index,x,y,z
5952,30,30-right_hand-0,right_hand,0,NaN,NaN,NaN
5953,30,30-right_hand-1,right_hand,1,NaN,NaN,NaN
5954,30,30-right_hand-2,right_hand,2,NaN,NaN,NaN
5955,30,30-right_hand-3,right_hand,3,NaN,NaN,NaN
5956,30,30-right_hand-4,right_hand,4,NaN,NaN,NaN
...,...,...,...,...,...,...,...
11941,41,41-right_hand-16,right_hand,16,NaN,NaN,NaN
11942,41,41-right_hand-17,right_hand,17,NaN,NaN,NaN
11943,41,41-right_hand-18,right_hand,18,NaN,NaN,NaN
11944,41,41-right_hand-19,right_hand,19,NaN,NaN,NaN


In [7]:
y.participant_id.value_counts()

49445    4968
61333    4900
36257    4896
16069    4848
26734    4841
55372    4826
2044     4810
37779    4782
32319    4753
29302    4722
22343    4677
53618    4656
37055    4648
28656    4563
62590    4563
34503    4545
27610    4275
25571    3865
18796    3502
4718     3499
30680    3338
Name: participant_id, dtype: int64

In [77]:
y.loc[y.participant_id == 26734]

,path,participant_id,sequence_id,sign
0,train_landmark_files/26734/1000035562.parquet,26734,1000035562,blow
5,train_landmark_files/26734/1000241583.parquet,26734,1000241583,duck
6,train_landmark_files/26734/1000255522.parquet,26734,1000255522,minemy
13,train_landmark_files/26734/1000661926.parquet,26734,1000661926,mitten
18,train_landmark_files/26734/1001145816.parquet,26734,1001145816,yellow
...,...,...,...,...
94419,train_landmark_files/26734/997396207.parquet,26734,997396207,orange
94440,train_landmark_files/26734/998617896.parquet,26734,998617896,finish
94447,train_landmark_files/26734/998892578.parquet,26734,998892578,wet
94456,train_landmark_files/26734/999099088.parquet,26734,999099088,tooth


In [78]:
y.loc[y.sign == 'blow'].loc[y.participant_id == 26734]

,path,participant_id,sequence_id,sign
0,train_landmark_files/26734/1000035562.parquet,26734,1000035562,blow
218,train_landmark_files/26734/1008843441.parquet,26734,1008843441,blow
3272,train_landmark_files/26734/1134422341.parquet,26734,1134422341,blow
11402,train_landmark_files/26734/1469847505.parquet,26734,1469847505,blow
14603,train_landmark_files/26734/1601834904.parquet,26734,1601834904,blow
22215,train_landmark_files/26734/1915935243.parquet,26734,1915935243,blow
29597,train_landmark_files/26734/2219113522.parquet,26734,2219113522,blow
38270,train_landmark_files/26734/2566139942.parquet,26734,2566139942,blow
50516,train_landmark_files/26734/3066953847.parquet,26734,3066953847,blow
62669,train_landmark_files/26734/356885486.parquet,26734,356885486,blow
